<a href="https://colab.research.google.com/github/krisograbek/text-preprocessing/blob/main/PythonTextPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In the notebook, you can find a collection of Text Preprocessing steps that are common in a variety of NLP tasks. However, the list is probably not complete. 

It contains:
 - Text Cleaning steps
 - Text Normalization steps
 - Sources of information

It does NOT contain:
 - Word Embeddings
 - Bag of Words
 - TF-IDF

In [ ]:
! pip install -q unidecode pyspellchecker autocorrect

# Table of Contents

1. [Lowercasing](#lowercase)
2. [Punctuation Removal](#removal)
    - [str.translate()](#rmvl-translate)
    - [Regular Expressions](#rmvl-re)
      - [re.compile()](#rmvl-re-compile)
      - [re.escape()](#rmvl-re-escape)
      - [Remove not words and not whitespaces](#rmvl-re-not-words)
    - [Excluding Punctuation](#rmvl-exclude)
    - [String Replace](#rmvl-str-replace)
3. [Numbers Removal](#removal-nb)
    - [str.translate()](#rmvl-nb-tr)
    - [Regular Expressions](#rmvl-nb-re)
    - [join with not digit](#rmvl-nb-not-digit)
    - [join with not alpha](#rmvl-nb-not-alpha)
4. [HTML Tags Removal](#html-rmvl)
    - [Regular Expressions](#html-rmvl-re)
    - [Beautiful Soup](#html-rmvl-bs)
5. [URL Removal](#url-rmvl)
    - [Regular Expressions](#url-rmvl-re)
6. [Newlines / spaces / tabs Removal](#spaces-rmvl)
    - [String split](#spaces-rmvl-split)
    - [Regular Expressions](#spaces-rmvl-re)
        - [re.sub()](#spaces-rmvl-re)
        - [re.findall()](#spaces-rmvl-re-findall)
7. [Emojis Removal](#emoji-rmvl)
    - [Regular Expressions](#emoji-rmvl-re)
8. [Replacing accented characters](#accented)
    - [Unidecode](#accented-unidecode)
9. [Spelling corrections](#spell)
    - [SpellChecker](#spell-checker)
    - [Autocorrect](#spell-autocorrect)
    - [TextBlob](#spell-tb)
9. [Tokenization](#tknz)
    - [Regular Expressions](#tknz-re)
    - [NLTK](#tknz-nltk)
    - [SpaCy](#tknz-spacy)
    - [Gensim](#tknz-gensim)
    - [Comparision of the methods](#tknz-compare)
10. [Sentence Tokenization](#tknz-sents)
    - [Regular Expressions](#tknz-sents-re)
    - [NLTK](#tknz-sents-nltk)
    - [SpaCy](#tknz-sents-spacy)
11. [Stopwords Removal](#stopwords)
    - [NLTK](#stopwords-nltk)
    - [SpaCy](#stopwords-spacy)
    - [Gensim](#stopwords-gensim)
    - [Comparision of the methods](#stopwords-compare)
    - [Comparision of stopwords lists](#stopwords-compare2)
12. [Lemmatization](#lemma)
    - [NLTK](#lemma-nltk)
    - [SpaCy](#lemma-spacy)
    - [TextBlob](#lemma-tb)
    - [Adding Tags](#lemma-tags)
        - [NLTK](#lemma-tags-nltk)
        - [TextBlob](#lemma-tags-tb)
    - [Comparision of the methods](#lemma-compare)
13. [Stemming](#stem)
    - [PorterStemmer](#stem-ps)
    - [SnowballStemmer](#stem-sno)


In [ ]:
import string
import re
import nltk
import spacy

## Lowercasing <a name="lowercase"></a>

In [ ]:
text_with_upper = "Hello, POLAND is a Very BeautiFul CountrY!"
text_with_upper.lower()

'hello, poland is a very beautiful country!'

## Punctuation Removal <a name="removal"></a>

### **Most of them use `string.punctuation`**



Based on the [StackOverflow question](https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string)

I ordered the solutions from the fastest to the slowest

In [ ]:
puncts_text = "HI!!! I overuse ... !(@*punctuations +-*/ and*(& other signs __*(&!!"

### using `str.translate()` <a name="rmvl-translate"></a>
Probably the fastest way

In [ ]:
puncts_text.translate(str.maketrans('', '', string.punctuation))

'HI I overuse  punctuations  and other signs '

### Using `re` <a name="rmvl-re"></a>

`re` stands for Regular Expressions

#### Method 1 with string.punctuation <a name="rmvl-re-compile"></a>

In [ ]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
regex.sub('', puncts_text)

'HI I overuse  punctuations  and other signs '

#### Method 1 in a single line <a name="rmvl-re-compile"></a>

In [ ]:
re.sub(f'[{re.escape(string.punctuation)}]','', puncts_text)

'HI I overuse  punctuations  and other signs '

#### Method 2 <a name="rmvl-re-not-words"></a>

Removes **not words** and **not spaces**

Note: `re` treats underscore as a word so its results are different

In [ ]:
re.sub(r'[^\w\s]','', puncts_text)

'HI I overuse  punctuations  and other signs __'

### Excluding string.punctuation <a name="rmvl-exclude"></a>

In [ ]:
# a slower solution
exclude = set(string.punctuation)
"".join(ch for ch in puncts_text if ch not in exclude)

'HI I overuse  punctuations  and other signs '

### Using `str.replace()` <a name="rmvl-str-replace"></a>

In [ ]:
clean_text = puncts_text
for c in string.punctuation:
    clean_text = clean_text.replace(c, "")
clean_text

'HI I overuse  punctuations  and other signs '

## Numbers Removal <a name="removal-nb"></a>



In [ ]:
text_numbers = '12abcd405'

### Using `str.translate` with `string.digits` <a name="rmvl-nb-tr"></a>

In [ ]:
text_numbers.translate(str.maketrans('', '', string.digits))

'abcd'

### Using `re` <a name="rmvl-nb-re"></a>

In [ ]:
re.sub(r'\d+', '', text_numbers)

'abcd'

In [ ]:
re.sub(r'[0-9]+', '', text_numbers)

'abcd'

### Using `join()` and NOT `isdigit()` <a name="rmvl-nb-not-digit"></a>

In [ ]:
"".join(i for i in text_numbers if not i.isdigit())

'abcd'

### Using `join()` and `isalpha()` <a name="rmvl-nb-not-alpha"></a>

This actually isn't the correct solution, because `isalpha()` is True only for letters.

In [ ]:
"".join(i for i in text_numbers if i.isalpha())

'abcd'

## HTML Tags Removal <a name="html-rmvl"></a>

Solutions from [Stack Overflow](https://stackoverflow.com/questions/753052/strip-html-from-strings-in-python)

In [ ]:
html_text = """<tr class="color-5 negri a-bottom">
<td class="a-center" width="11%"><div style="min-width: 80px">3-Pointers</div></td>
<td><div class="left" style="min-width: 120px; max-width:175px; width: 57%">
<div class="left margen-l2">Player</div>
<div class="right"> Team</div>
</div>
</td>
<td><div style="min-width: 60px; ">Season</div></td>
<td><div class="">W/L Game</div>
</td>
</tr>"""


### Using `re` <a name="html-rmvl-re"></a>

 - begin with tag opening '<'
 - then not '<'
 - not '<' at least once

In [ ]:
re.sub('<[^<]+?>', '', html_text)

'\n3-Pointers\n\nPlayer\n Team\n\n\nSeason\nW/L Game\n\n'

### Using `BeautifulSoup` <a name="html-rmvl-bs"></a>

 - `get_text()` removes HTML tags
 - `strip = True` removes whitespaces and newlines

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(html_text, 'html.parser')
soup.get_text(",", strip=True)

'3-Pointers,Player,Team,Season,W/L Game'

### Extreme example

In [ ]:
html_comment = "<img<!-- --> src=x onerror=alert(1);//><!-- -->"

Both solutions fail

In [ ]:
re.sub('<[^<]+?>', '', html_comment)

'<img src=x onerror=alert(1);//>'

In [ ]:
soup = BeautifulSoup(html_comment, 'html.parser')
soup.get_text(",", strip=True)

'src=x onerror=alert(1);//>'

In [ ]:
tag_re = re.compile(r'(<!--.*?-->|<[^>]*>)')
no_tags = tag_re.sub('', html_comment)
no_tags

' src=x onerror=alert(1);//>'

In [ ]:
import html

html.escape(no_tags)

' src=x onerror=alert(1);//&gt;'

## URL Removal <a name="url-rmvl"></a>

From this [StackOverflow Question](https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python/11332580)

In [ ]:
text_url = """text1
text2
http://url.com/bla1/blah1/
text3
text4
http://url.com/bla2/blah2/
text5
text6
http://url.com/bla3/blah3/"""

### Using `re` <a name="url-rmvl-re"></a>

In [ ]:
re.sub(r'http\S+', '', text_url)

'text1\ntext2\n\ntext3\ntext4\n\ntext5\ntext6\n'

## Newlines, spaces and tabs removal <a name="spaces-rmvl"></a>

### Using `str.split()` and `join()` <a name="spaces-rmvl-split"></a>

In [ ]:
my_str="I want to Remove all white \t\n\n\r spaces, new lines \n and tabs \t"
" ".join(my_str.split())

'I want to Remove all white spaces, new lines and tabs'

### Using `re` <a name="spaces-rmvl-re"></a>

 - **`\s` stands for whitespace character, equivalent to `[ \n\r\t\f]`**
 - **`\S` stands for not whitespace character, equivalent to `[^\s]`**

In [ ]:
re.sub('\s+', ' ', my_str)

'I want to Remove all white spaces, new lines and tabs '

In [ ]:
re.sub('[^\S]+', ' ', my_str)

'I want to Remove all white spaces, new lines and tabs '

In [ ]:
re.sub('[\t\n\r\f ]+', ' ', my_str)

'I want to Remove all white spaces, new lines and tabs '

### Using `re.findall()` <a name="spaces-rmvl-re-findall"></a>

Taken from [StackOverflow](https://stackoverflow.com/questions/4697882/how-can-i-find-all-matches-to-a-regular-expression-in-python)

NOTE: I believe this approach is slower than the one with `re.sub()`. 

In [ ]:
match = re.findall('[\w]+ ', my_str)
"".join(match)

'I want to Remove all white new lines and tabs '

## Emojis Removal <a name="emoji-rmvl"></a>

Found [here](https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b#gistcomment-3315605)

In [ ]:
text_emojis = u"Hi 🤔 How is your 🙈 and 😌. Have a nice weekend 💕👭👙\U0001F600\U0001F300"
text_emojis

'Hi 🤔 How is your 🙈 and 😌. Have a nice weekend 💕👭👙😀🌀'

### Using `re` <a name="emoji-rmvl-re"></a>

In [ ]:
emojis_pattern = re.compile(pattern="["
                    u"\U0001F600-\U0001F64F"  # emoticons
                    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                    u"\U0001F680-\U0001F6FF"  # transport & map symbols
                    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                    u"\U00002500-\U00002BEF"  # chinese char
                    u"\U00002702-\U000027B0"
                    u"\U00002702-\U000027B0"
                    u"\U000024C2-\U0001F251"
                    u"\U0001f926-\U0001f937"
                    u"\U00010000-\U0010ffff"
                    u"\u2640-\u2642"
                    u"\u2600-\u2B55"
                    u"\u200d"
                    u"\u23cf"
                    u"\u23e9"
                    u"\u231a"
                    u"\ufe0f"  # dingbats
                    u"\u3030"
                "]+", flags = re.UNICODE)

emojis_pattern.sub(r'', text_emojis)

'Hi  How is your  and . Have a nice weekend '

## Replacing Accented Characters <a name="accented"></a>

### Using `unidecode` <a name="accented-unidecode"></a>

In [ ]:
import unidecode
text_accented = "Málaga, àéêöhello. Polish: ńŃćĆśŚęąóżŻźŹ letters. German üöäöß letters"

unidecode.unidecode(text_accented)

'Malaga, aeeohello. Polish: nNcCsSeaozZzZ letters. German uoaoss letters'

## Spelling corrections <a name="spell"></a>

Solutions from [StackOverflow](https://stackoverflow.com/questions/13928155/spell-checker-for-python)

### Using `spellchecker` <a name="spell-checker"></a>

In [ ]:
from spellchecker import SpellChecker

spell = SpellChecker()

def correct_spelling(text):
    corrected_text = list()
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        next_word = word
        if word in misspelled_words:
            next_word = spell.correction(word)
        corrected_text.append(next_word)
    
    return " ".join(corrected_text)

text_misspelled = "I realli needt smoe corection. This sentnce has mispelled wirds"
correct_spelling(text_misspelled)

'I really need some corrections This sentence has misspelled words'

### Using `autocorrect`<a name="spell-autocorrect"></a>

In [ ]:
from autocorrect import Speller

speller = Speller(lang='en')

print(speller(text_misspelled))

I really need some correction. This sentence has misspelled words


### Using `textblob` <a name="spell-tb"></a>

Found [here](https://www.geeksforgeeks.org/python-textblob-correct-method/)

In [ ]:
from textblob import TextBlob

TextBlob(text_misspelled).correct()

TextBlob("I really need some correction. His sentence has dispelled words")

### Create a longer text

Beginning of the Metamorphosis by Franz Kafka

In [ ]:
text_lines = """One morning, when Gregor Samsa woke from troubled dreams,
 he found himself transformed in his bed into a horrible vermin.  He lay 
 on his armour-like back, and if he lifted his head a little he could see his 
 brown belly, slightly domed and divided by arches into stiff sections.  
 The bedding was hardly able to cover it and seemed ready to slide off any 
 moment.  His many legs, pitifully thin compared with the size of the rest of 
 him, waved about helplessly as he looked.  "What's happened to me?" he thought.  
 It wasn't a dream.  His room, a proper human room although a little too small, 
 lay peacefully between its four familiar walls.  A collection of textile 
 samples lay spread out on the table - Samsa was a travelling salesman - and 
 above it there hung a picture that he had recently cut out of an illustrated 
 magazine and housed in a nice, gilded frame.  It showed a lady fitted out with 
 a fur hat and fur boa who sat upright, raising a heavy fur muff that covered 
 the whole of her lower arm towards the viewer.  Gregor then turned to look out 
 the window at the dull weather."""
text_lines

'One morning, when Gregor Samsa woke from troubled dreams,\n he found himself transformed in his bed into a horrible vermin.  He lay \n on his armour-like back, and if he lifted his head a little he could see his \n brown belly, slightly domed and divided by arches into stiff sections.  \n The bedding was hardly able to cover it and seemed ready to slide off any \n moment.  His many legs, pitifully thin compared with the size of the rest of \n him, waved about helplessly as he looked.  "What\'s happened to me?" he thought.  \n It wasn\'t a dream.  His room, a proper human room although a little too small, \n lay peacefully between its four familiar walls.  A collection of textile \n samples lay spread out on the table - Samsa was a travelling salesman - and \n above it there hung a picture that he had recently cut out of an illustrated \n magazine and housed in a nice, gilded frame.  It showed a lady fitted out with \n a fur hat and fur boa who sat upright, raising a heavy fur muff tha

## Tokenize Words <a name="tknz"></a>

### Using `re` <a name="tknz-re"></a>

Return the list of words

In [ ]:
re_tokens = re.findall('[\w]+', text_lines)
print(len(re_tokens))
print(re_tokens)

200
['One', 'morning', 'when', 'Gregor', 'Samsa', 'woke', 'from', 'troubled', 'dreams', 'he', 'found', 'himself', 'transformed', 'in', 'his', 'bed', 'into', 'a', 'horrible', 'vermin', 'He', 'lay', 'on', 'his', 'armour', 'like', 'back', 'and', 'if', 'he', 'lifted', 'his', 'head', 'a', 'little', 'he', 'could', 'see', 'his', 'brown', 'belly', 'slightly', 'domed', 'and', 'divided', 'by', 'arches', 'into', 'stiff', 'sections', 'The', 'bedding', 'was', 'hardly', 'able', 'to', 'cover', 'it', 'and', 'seemed', 'ready', 'to', 'slide', 'off', 'any', 'moment', 'His', 'many', 'legs', 'pitifully', 'thin', 'compared', 'with', 'the', 'size', 'of', 'the', 'rest', 'of', 'him', 'waved', 'about', 'helplessly', 'as', 'he', 'looked', 'What', 's', 'happened', 'to', 'me', 'he', 'thought', 'It', 'wasn', 't', 'a', 'dream', 'His', 'room', 'a', 'proper', 'human', 'room', 'although', 'a', 'little', 'too', 'small', 'lay', 'peacefully', 'between', 'its', 'four', 'familiar', 'walls', 'A', 'collection', 'of', 'textile

### Using `nltk` <a name="tknz-nltk"></a>

In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

nltk_tokens = word_tokenize(text_lines)
print(len(nltk_tokens), nltk_tokens)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
224 ['One', 'morning', ',', 'when', 'Gregor', 'Samsa', 'woke', 'from', 'troubled', 'dreams', ',', 'he', 'found', 'himself', 'transformed', 'in', 'his', 'bed', 'into', 'a', 'horrible', 'vermin', '.', 'He', 'lay', 'on', 'his', 'armour-like', 'back', ',', 'and', 'if', 'he', 'lifted', 'his', 'head', 'a', 'little', 'he', 'could', 'see', 'his', 'brown', 'belly', ',', 'slightly', 'domed', 'and', 'divided', 'by', 'arches', 'into', 'stiff', 'sections', '.', 'The', 'bedding', 'was', 'hardly', 'able', 'to', 'cover', 'it', 'and', 'seemed', 'ready', 'to', 'slide', 'off', 'any', 'moment', '.', 'His', 'many', 'legs', ',', 'pitifully', 'thin', 'compared', 'with', 'the', 'size', 'of', 'the', 'rest', 'of', 'him', ',', 'waved', 'about', 'helplessly', 'as', 'he', 'looked', '.', '``', 'What', "'s", 'happened', 'to', 'me', '?', "''", 'he', 'thought', '.', 'It', 'was', "n't", 'a', 'dream', '.', 'His

### Using `spaCy` <a name="tknz-spacy"></a>

In [ ]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(text_lines)
spacy_tokens = list([token for token in doc])
print(len(spacy_tokens))
print(spacy_tokens)

247
[One, morning, ,, when, Gregor, Samsa, woke, from, troubled, dreams, ,, 
 , he, found, himself, transformed, in, his, bed, into, a, horrible, vermin, .,  , He, lay, 
 , on, his, armour, -, like, back, ,, and, if, he, lifted, his, head, a, little, he, could, see, his, 
 , brown, belly, ,, slightly, domed, and, divided, by, arches, into, stiff, sections, .,  
 , The, bedding, was, hardly, able, to, cover, it, and, seemed, ready, to, slide, off, any, 
 , moment, .,  , His, many, legs, ,, pitifully, thin, compared, with, the, size, of, the, rest, of, 
 , him, ,, waved, about, helplessly, as, he, looked, .,  , ", What, 's, happened, to, me, ?, ", he, thought, .,  
 , It, was, n't, a, dream, .,  , His, room, ,, a, proper, human, room, although, a, little, too, small, ,, 
 , lay, peacefully, between, its, four, familiar, walls, .,  , A, collection, of, textile, 
 , samples, lay, spread, out, on, the, table, -, Samsa, was, a, travelling, salesman, -, and, 
 , above, it, there, hung, a, pic

### Using `gensim` <a name="tknz-gensim"></a>

In [ ]:
from gensim.utils import tokenize

gensim_tokens = list(tokenize(text_lines))
print(len(gensim_tokens))
print(gensim_tokens)

200
['One', 'morning', 'when', 'Gregor', 'Samsa', 'woke', 'from', 'troubled', 'dreams', 'he', 'found', 'himself', 'transformed', 'in', 'his', 'bed', 'into', 'a', 'horrible', 'vermin', 'He', 'lay', 'on', 'his', 'armour', 'like', 'back', 'and', 'if', 'he', 'lifted', 'his', 'head', 'a', 'little', 'he', 'could', 'see', 'his', 'brown', 'belly', 'slightly', 'domed', 'and', 'divided', 'by', 'arches', 'into', 'stiff', 'sections', 'The', 'bedding', 'was', 'hardly', 'able', 'to', 'cover', 'it', 'and', 'seemed', 'ready', 'to', 'slide', 'off', 'any', 'moment', 'His', 'many', 'legs', 'pitifully', 'thin', 'compared', 'with', 'the', 'size', 'of', 'the', 'rest', 'of', 'him', 'waved', 'about', 'helplessly', 'as', 'he', 'looked', 'What', 's', 'happened', 'to', 'me', 'he', 'thought', 'It', 'wasn', 't', 'a', 'dream', 'His', 'room', 'a', 'proper', 'human', 'room', 'although', 'a', 'little', 'too', 'small', 'lay', 'peacefully', 'between', 'its', 'four', 'familiar', 'walls', 'A', 'collection', 'of', 'textile

### Tokenization Comparision <a name="tknz-compare"></a>

 - it seems that `gensim` uses the same `re` function, that we showed above. Both returned only words
 - `nltk` and `spacy` return also punctuations
 - `spacy` treats a whitespace as a token if there is a double whitespace. In our text each sentence-ending dot is followed by double whitespace. We could clean this but at least we see that `spacy` behaves differently
 - **not or n't** contraction gives different results. `spacy` and `nltk` splits *wasn't* to *was* and *n't*, whereas `re` and `gensim` to *wasn* and *t*.
 - when there is a hyphen between words, we get 3 different results. Our example is armour-like. `nltk` returns **a single token**: *armour-like*, `re` and `gensim` return **two tokens**: *armour* and *like*. `spacy` returns **three tokens**: *armour*, *-*, and *like*
 - `nltk` converts quotation marks. Quote opening: **``**, quote closing **' '**

## Tokenize Sentences <a name="tknz-sents"></a>

### Using `re` <a name="tknz-sents-re"></a>

In [ ]:
re_sentences = re.compile('[.?!]').split(text_lines)
len(re_sentences), re_sentences

(12,
 ['One morning, when Gregor Samsa woke from troubled dreams,\n he found himself transformed in his bed into a horrible vermin',
  '  He lay \n on his armour-like back, and if he lifted his head a little he could see his \n brown belly, slightly domed and divided by arches into stiff sections',
  '  \n The bedding was hardly able to cover it and seemed ready to slide off any \n moment',
  '  His many legs, pitifully thin compared with the size of the rest of \n him, waved about helplessly as he looked',
  '  "What\'s happened to me',
  '" he thought',
  "  \n It wasn't a dream",
  '  His room, a proper human room although a little too small, \n lay peacefully between its four familiar walls',
  '  A collection of textile \n samples lay spread out on the table - Samsa was a travelling salesman - and \n above it there hung a picture that he had recently cut out of an illustrated \n magazine and housed in a nice, gilded frame',
  '  It showed a lady fitted out with \n a fur hat and fu

### Using `nltk` <a name="tknz-sents-nltk"></a>

In [ ]:
from nltk.tokenize import sent_tokenize

nltk_sentences = sent_tokenize(text_lines)
print(len(nltk_sentences), nltk_sentences)

11 ['One morning, when Gregor Samsa woke from troubled dreams,\n he found himself transformed in his bed into a horrible vermin.', 'He lay \n on his armour-like back, and if he lifted his head a little he could see his \n brown belly, slightly domed and divided by arches into stiff sections.', 'The bedding was hardly able to cover it and seemed ready to slide off any \n moment.', 'His many legs, pitifully thin compared with the size of the rest of \n him, waved about helplessly as he looked.', '"What\'s happened to me?"', 'he thought.', "It wasn't a dream.", 'His room, a proper human room although a little too small, \n lay peacefully between its four familiar walls.', 'A collection of textile \n samples lay spread out on the table - Samsa was a travelling salesman - and \n above it there hung a picture that he had recently cut out of an illustrated \n magazine and housed in a nice, gilded frame.', 'It showed a lady fitted out with \n a fur hat and fur boa who sat upright, raising a he

### Using `spaCy` <a name="tknz-sents-spacy"></a>

In [ ]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(text_lines)
spacy_sentences = list([sent for sent in doc.sents])
len(spacy_sentences), spacy_sentences

(13, [One morning, when Gregor Samsa woke from troubled dreams,
   he found himself transformed in his bed into a horrible vermin.  , He lay 
   on his armour-like back, and if he lifted his head a little he could see his 
   brown belly, slightly domed and divided by arches into stiff sections.  
   , The bedding was hardly able to cover it and seemed ready to slide off any 
   moment.  , His many legs, pitifully thin compared with the size of the rest of 
   him, waved about helplessly as he looked.  , "What's happened to me?, " he thought.  
   , It wasn't a dream.  , His room, a proper human room although a little too small, 
   lay peacefully between its four familiar walls.  , A collection of textile 
   samples lay spread out on the table - Samsa was a travelling salesman - and 
   above it there, hung a picture that he had recently cut out of an illustrated 
   magazine and housed in a nice, gilded frame.  , It showed a lady fitted out with 
   a fur hat and fur boa who sat upr

## Stopwords Removal <a name="stopwords"></a>

We'll ignore punctuations. Tokenization step using `re` gives us exactly that.

### Using `nltk` <a name="stopwords-nltk"></a>

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words_nltk = stopwords.words('english')
# print(len(stop_words_nltk),stop_words_nltk)

filtered_nltk = [word for word in re_tokens if word.lower() not in stop_words_nltk]
print(len(filtered_nltk), filtered_nltk)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
106 ['One', 'morning', 'Gregor', 'Samsa', 'woke', 'troubled', 'dreams', 'found', 'transformed', 'bed', 'horrible', 'vermin', 'lay', 'armour', 'like', 'back', 'lifted', 'head', 'little', 'could', 'see', 'brown', 'belly', 'slightly', 'domed', 'divided', 'arches', 'stiff', 'sections', 'bedding', 'hardly', 'able', 'cover', 'seemed', 'ready', 'slide', 'moment', 'many', 'legs', 'pitifully', 'thin', 'compared', 'size', 'rest', 'waved', 'helplessly', 'looked', 'happened', 'thought', 'dream', 'room', 'proper', 'human', 'room', 'although', 'little', 'small', 'lay', 'peacefully', 'four', 'familiar', 'walls', 'collection', 'textile', 'samples', 'lay', 'spread', 'table', 'Samsa', 'travelling', 'salesman', 'hung', 'picture', 'recently', 'cut', 'illustrated', 'magazine', 'housed', 'nice', 'gilded', 'frame', 'showed', 'lady', 'fitted', 'fur', 'hat', 'fur', 'boa', 'sat', 'upright', 'ra

### Using `spaCy` <a name="stopwords-spacy"></a>

In [ ]:
stop_words_spacy = nlp.Defaults.stop_words

filtered_spacy = [word for word in re_tokens if word.lower() not in stop_words_spacy]
print(len(filtered_spacy), filtered_spacy)

99 ['morning', 'Gregor', 'Samsa', 'woke', 'troubled', 'dreams', 'found', 'transformed', 'bed', 'horrible', 'vermin', 'lay', 'armour', 'like', 'lifted', 'head', 'little', 'brown', 'belly', 'slightly', 'domed', 'divided', 'arches', 'stiff', 'sections', 'bedding', 'hardly', 'able', 'cover', 'ready', 'slide', 'moment', 'legs', 'pitifully', 'thin', 'compared', 'size', 'rest', 'waved', 'helplessly', 'looked', 's', 'happened', 'thought', 'wasn', 't', 'dream', 'room', 'proper', 'human', 'room', 'little', 'small', 'lay', 'peacefully', 'familiar', 'walls', 'collection', 'textile', 'samples', 'lay', 'spread', 'table', 'Samsa', 'travelling', 'salesman', 'hung', 'picture', 'recently', 'cut', 'illustrated', 'magazine', 'housed', 'nice', 'gilded', 'frame', 'showed', 'lady', 'fitted', 'fur', 'hat', 'fur', 'boa', 'sat', 'upright', 'raising', 'heavy', 'fur', 'muff', 'covered', 'lower', 'arm', 'viewer', 'Gregor', 'turned', 'look', 'window', 'dull', 'weather']


### Using `gensim` <a name="stopwords-gensim"></a>

In [ ]:
from gensim.parsing.preprocessing import STOPWORDS

stop_words_gensim = STOPWORDS

filtered_gensim = [word for word in re_tokens if word.lower() not in stop_words_gensim]
print(len(filtered_gensim), filtered_gensim)

97 ['morning', 'Gregor', 'Samsa', 'woke', 'troubled', 'dreams', 'transformed', 'bed', 'horrible', 'vermin', 'lay', 'armour', 'like', 'lifted', 'head', 'little', 'brown', 'belly', 'slightly', 'domed', 'divided', 'arches', 'stiff', 'sections', 'bedding', 'hardly', 'able', 'cover', 'ready', 'slide', 'moment', 'legs', 'pitifully', 'compared', 'size', 'rest', 'waved', 'helplessly', 'looked', 's', 'happened', 'thought', 'wasn', 't', 'dream', 'room', 'proper', 'human', 'room', 'little', 'small', 'lay', 'peacefully', 'familiar', 'walls', 'collection', 'textile', 'samples', 'lay', 'spread', 'table', 'Samsa', 'travelling', 'salesman', 'hung', 'picture', 'recently', 'cut', 'illustrated', 'magazine', 'housed', 'nice', 'gilded', 'frame', 'showed', 'lady', 'fitted', 'fur', 'hat', 'fur', 'boa', 'sat', 'upright', 'raising', 'heavy', 'fur', 'muff', 'covered', 'lower', 'arm', 'viewer', 'Gregor', 'turned', 'look', 'window', 'dull', 'weather']


Another method from gensim using the `remove_stopwords` function.

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords

filtered_sentence_gensim = remove_stopwords(text_lines)
print(len(filtered_sentence_gensim), filtered_sentence_gensim)

713 One morning, Gregor Samsa woke troubled dreams, transformed bed horrible vermin. He lay armour-like back, lifted head little brown belly, slightly domed divided arches stiff sections. The bedding hardly able cover ready slide moment. His legs, pitifully compared size rest him, waved helplessly looked. "What's happened me?" thought. It wasn't dream. His room, proper human room little small, lay peacefully familiar walls. A collection textile samples lay spread table - Samsa travelling salesman - hung picture recently cut illustrated magazine housed nice, gilded frame. It showed lady fitted fur hat fur boa sat upright, raising heavy fur muff covered lower arm viewer. Gregor turned look window dull weather.


### Comparing results <a name="stopwords-compare"></a>

**`nltk` vs `spacy`**

In [ ]:
list(set(filtered_nltk) ^ set(filtered_spacy))

['could',
 'although',
 'seemed',
 'many',
 't',
 'back',
 'see',
 's',
 'four',
 'towards',
 'wasn',
 'One',
 'whole']

**`nltk` vs `gensim`**

In [ ]:
list(set(filtered_nltk) ^ set(filtered_gensim))

['could',
 'although',
 'seemed',
 'found',
 'many',
 't',
 'back',
 'see',
 's',
 'thin',
 'four',
 'towards',
 'wasn',
 'One',
 'whole']

**`gensim` vs `spacy`**

In [ ]:
list(set(filtered_gensim) ^ set(filtered_spacy))

['found', 'thin']

### Comparing stopwords lists <a name="stopwords-compare2"></a>

Lists of words

In [ ]:
# print("NLTK stopwords",stop_words_nltk)
# print("Spacy stopwords",stop_words_spacy)
# print("Gensim stopwords",stop_words_gensim)

Comparing length

In [ ]:
print("NLTK stopwords len", len(stop_words_nltk))
print("Spacy stopwords len", len(stop_words_spacy))
print("Gensim stopwords len", len(stop_words_gensim))

NLTK stopwords len 179
Spacy stopwords len 326
Gensim stopwords len 337


## Lemmatization <a name="lemma"></a>

For lemmatization we'll ignore punctuations. Tokenization step using `re` gives us exactly that.

### Using `nltk` <a name="lemma-nltk"></a>

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

nltk_lemmatizer = WordNetLemmatizer()
nltk_lemmas = [nltk_lemmatizer.lemmatize(w) for w in re_tokens]
print(nltk_lemmas)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
['One', 'morning', 'when', 'Gregor', 'Samsa', 'woke', 'from', 'troubled', 'dream', 'he', 'found', 'himself', 'transformed', 'in', 'his', 'bed', 'into', 'a', 'horrible', 'vermin', 'He', 'lay', 'on', 'his', 'armour', 'like', 'back', 'and', 'if', 'he', 'lifted', 'his', 'head', 'a', 'little', 'he', 'could', 'see', 'his', 'brown', 'belly', 'slightly', 'domed', 'and', 'divided', 'by', 'arch', 'into', 'stiff', 'section', 'The', 'bedding', 'wa', 'hardly', 'able', 'to', 'cover', 'it', 'and', 'seemed', 'ready', 'to', 'slide', 'off', 'any', 'moment', 'His', 'many', 'leg', 'pitifully', 'thin', 'compared', 'with', 'the', 'size', 'of', 'the', 'rest', 'of', 'him', 'waved', 'about', 'helplessly', 'a', 'he', 'looked', 'What', 's', 'happened', 'to', 'me', 'he', 'thought', 'It', 'wasn', 't', 'a', 'dream', 'His', 'room', 'a', 'proper', 'human', 'room', 'although', 'a', 'little', 'too', 'small

### Using `spaCy` <a name="lemma-spacy"></a>

[spacy documentation](https://spacy.io/api/lemmatizer)

In [ ]:
# spacy_text = " ".join([token.text for token in spacy_tokens])
text_from_re_tokens = " ".join([word for word in re_tokens])
# spacy_lemmas = [word.lemma_ for word in nlp(text_lines)]
spacy_lemmas = [word.lemma_ for word in nlp(text_from_re_tokens)]
print(len(spacy_lemmas), spacy_lemmas)

200 ['one', 'morning', 'when', 'Gregor', 'Samsa', 'wake', 'from', 'troubled', 'dream', '-PRON-', 'find', '-PRON-', 'transform', 'in', '-PRON-', 'bed', 'into', 'a', 'horrible', 'vermin', '-PRON-', 'lie', 'on', '-PRON-', 'armour', 'like', 'back', 'and', 'if', '-PRON-', 'lift', '-PRON-', 'head', 'a', 'little', '-PRON-', 'could', 'see', '-PRON-', 'brown', 'belly', 'slightly', 'dome', 'and', 'divide', 'by', 'arch', 'into', 'stiff', 'section', 'the', 'bedding', 'be', 'hardly', 'able', 'to', 'cover', '-PRON-', 'and', 'seem', 'ready', 'to', 'slide', 'off', 'any', 'moment', '-PRON-', 'many', 'leg', 'pitifully', 'thin', 'compare', 'with', 'the', 'size', 'of', 'the', 'rest', 'of', '-PRON-', 'wave', 'about', 'helplessly', 'as', '-PRON-', 'look', 'what', 's', 'happen', 'to', '-PRON-', '-PRON-', 'think', '-PRON-', 'wasn', 't', 'a', 'dream', '-PRON-', 'room', 'a', 'proper', 'human', 'room', 'although', 'a', 'little', 'too', 'small', 'lie', 'peacefully', 'between', '-PRON-', 'four', 'familiar', 'wall'

### Using `TextBlob` <a name="lemma-tb"></a>

In [ ]:
from textblob import TextBlob, Word

# create a TextBlob for our sentence
sent_tb = TextBlob(text_from_re_tokens)

# lemmatize each word
blob_lemmas = [word.lemmatize() for word in sent_tb.words]
print(len(blob_lemmas), blob_lemmas)

200 ['One', 'morning', 'when', 'Gregor', 'Samsa', 'woke', 'from', 'troubled', 'dream', 'he', 'found', 'himself', 'transformed', 'in', 'his', 'bed', 'into', 'a', 'horrible', 'vermin', 'He', 'lay', 'on', 'his', 'armour', 'like', 'back', 'and', 'if', 'he', 'lifted', 'his', 'head', 'a', 'little', 'he', 'could', 'see', 'his', 'brown', 'belly', 'slightly', 'domed', 'and', 'divided', 'by', 'arch', 'into', 'stiff', 'section', 'The', 'bedding', 'wa', 'hardly', 'able', 'to', 'cover', 'it', 'and', 'seemed', 'ready', 'to', 'slide', 'off', 'any', 'moment', 'His', 'many', 'leg', 'pitifully', 'thin', 'compared', 'with', 'the', 'size', 'of', 'the', 'rest', 'of', 'him', 'waved', 'about', 'helplessly', 'a', 'he', 'looked', 'What', 's', 'happened', 'to', 'me', 'he', 'thought', 'It', 'wasn', 't', 'a', 'dream', 'His', 'room', 'a', 'proper', 'human', 'room', 'although', 'a', 'little', 'too', 'small', 'lay', 'peacefully', 'between', 'it', 'four', 'familiar', 'wall', 'A', 'collection', 'of', 'textile', 'sampl

### Adding POS Tags to `nltk` and `TextBlob` <a name="lemma-tags"></a>

By default, `nltk` and `TextBlob` treat every word as a noun. This is why words like "woke", "found", or "transformed" don't change after the lemmatization step. We can provide more information by adding the corresponding Part of Speech for each token. 

### Using `nltk` <a name="lemma-tags-nltk"></a>

 - we use `pos_tag()` to get tokens along with tags
 - we call the `lemmatize()` function with the second parameter, that is a tag

[Source](https://www.guru99.com/stemming-lemmatization-python-nltk.html)

In [ ]:
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from collections import defaultdict
nltk.download('averaged_perceptron_tagger')

tag_map_nltk = defaultdict(lambda : wn.NOUN)
tag_map_nltk['J'] = wn.ADJ
tag_map_nltk['V'] = wn.VERB
tag_map_nltk['R'] = wn.ADV

nltk_lemmas2 = [nltk_lemmatizer.lemmatize(token, tag_map_nltk[tag[0]]) for token, tag in pos_tag(re_tokens)]
print(nltk_lemmas2)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
['One', 'morning', 'when', 'Gregor', 'Samsa', 'wake', 'from', 'troubled', 'dream', 'he', 'find', 'himself', 'transform', 'in', 'his', 'bed', 'into', 'a', 'horrible', 'vermin', 'He', 'lay', 'on', 'his', 'armour', 'like', 'back', 'and', 'if', 'he', 'lift', 'his', 'head', 'a', 'little', 'he', 'could', 'see', 'his', 'brown', 'belly', 'slightly', 'domed', 'and', 'divide', 'by', 'arch', 'into', 'stiff', 'section', 'The', 'bedding', 'be', 'hardly', 'able', 'to', 'cover', 'it', 'and', 'seem', 'ready', 'to', 'slide', 'off', 'any', 'moment', 'His', 'many', 'leg', 'pitifully', 'thin', 'compare', 'with', 'the', 'size', 'of', 'the', 'rest', 'of', 'him', 'wave', 'about', 'helplessly', 'a', 'he', 'look', 'What', 's', 'happen', 'to', 'me', 'he', 'think', 'It', 'wasn', 't', 'a', 'dream', 'His', 'room', 'a', 'proper', 

### Using `TextBlob` <a name="lemma-tags-tb"></a>

 - we call `TextBlob(text).tags` to get tokens and tags
 - we call `word.lemmatize()` with the tag parameter

[Source](https://www.machinelearningplus.com/nlp/lemmatization-examples-python/#textbloblemmatizer)

In [ ]:
tag_map_tb = {  "J": 'a', # adjectives
                "N": 'n', # nouns
                "V": 'v', # verbs
                "R": 'r'} # adverbs

words_and_tags = [(w, tag_map_tb.get(pos[0], 'n')) for w, pos in sent_tb.tags]
blob_lemmas2 = [word.lemmatize(tag) for word, tag in words_and_tags]
print(len(blob_lemmas2), blob_lemmas2)

200 ['One', 'morning', 'when', 'Gregor', 'Samsa', 'wake', 'from', 'troubled', 'dream', 'he', 'find', 'himself', 'transform', 'in', 'his', 'bed', 'into', 'a', 'horrible', 'vermin', 'He', 'lay', 'on', 'his', 'armour', 'like', 'back', 'and', 'if', 'he', 'lift', 'his', 'head', 'a', 'little', 'he', 'could', 'see', 'his', 'brown', 'belly', 'slightly', 'domed', 'and', 'divide', 'by', 'arch', 'into', 'stiff', 'section', 'The', 'bedding', 'be', 'hardly', 'able', 'to', 'cover', 'it', 'and', 'seem', 'ready', 'to', 'slide', 'off', 'any', 'moment', 'His', 'many', 'leg', 'pitifully', 'thin', 'compare', 'with', 'the', 'size', 'of', 'the', 'rest', 'of', 'him', 'wave', 'about', 'helplessly', 'a', 'he', 'look', 'What', 's', 'happen', 'to', 'me', 'he', 'think', 'It', 'wasn', 't', 'a', 'dream', 'His', 'room', 'a', 'proper', 'human', 'room', 'although', 'a', 'little', 'too', 'small', 'lay', 'peacefully', 'between', 'it', 'four', 'familiar', 'wall', 'A', 'collection', 'of', 'textile', 'sample', 'lay', 'spre

### Comparing Lemmatizers <a name="lemma-compare"></a>

In [ ]:
list(set(nltk_lemmas) ^ set(blob_lemmas))

[]

In [ ]:
list(set(nltk_lemmas2) ^ set(blob_lemmas2))

[]

`nltk` and `TextBlob` return identical results.

Let's see what's changed after applying POS tags

In [ ]:
sorted(list(set(nltk_lemmas) ^ set(nltk_lemmas2)))

['be',
 'compare',
 'compared',
 'covered',
 'divide',
 'divided',
 'find',
 'fit',
 'fitted',
 'found',
 'gild',
 'gilded',
 'had',
 'hang',
 'happen',
 'happened',
 'have',
 'house',
 'housed',
 'hung',
 'lift',
 'lifted',
 'looked',
 'low',
 'lower',
 'raise',
 'raising',
 'sat',
 'seem',
 'seemed',
 'show',
 'showed',
 'sit',
 'think',
 'thought',
 'transform',
 'transformed',
 'turn',
 'turned',
 'wa',
 'wake',
 'wave',
 'waved',
 'woke']

`nltk` did a great job at turning past tense verbs to the present tense

## Stemming with `nltk` <a name="stem"></a>

Compare 2 types of Stemmers. [Source](https://stackoverflow.com/questions/24647400/what-is-the-best-stemming-method-in-python)

### Using `PorterStemmer()` <a name="stem-ps"></a>

In [ ]:
from nltk.stem import PorterStemmer

ps = PorterStemmer()

ps_stemms = [ps.stem(w) for w in re_tokens]
print(ps_stemms)

['one', 'morn', 'when', 'gregor', 'samsa', 'woke', 'from', 'troubl', 'dream', 'he', 'found', 'himself', 'transform', 'in', 'hi', 'bed', 'into', 'a', 'horribl', 'vermin', 'He', 'lay', 'on', 'hi', 'armour', 'like', 'back', 'and', 'if', 'he', 'lift', 'hi', 'head', 'a', 'littl', 'he', 'could', 'see', 'hi', 'brown', 'belli', 'slightli', 'dome', 'and', 'divid', 'by', 'arch', 'into', 'stiff', 'section', 'the', 'bed', 'wa', 'hardli', 'abl', 'to', 'cover', 'it', 'and', 'seem', 'readi', 'to', 'slide', 'off', 'ani', 'moment', 'hi', 'mani', 'leg', 'piti', 'thin', 'compar', 'with', 'the', 'size', 'of', 'the', 'rest', 'of', 'him', 'wave', 'about', 'helplessli', 'as', 'he', 'look', 'what', 's', 'happen', 'to', 'me', 'he', 'thought', 'It', 'wasn', 't', 'a', 'dream', 'hi', 'room', 'a', 'proper', 'human', 'room', 'although', 'a', 'littl', 'too', 'small', 'lay', 'peac', 'between', 'it', 'four', 'familiar', 'wall', 'A', 'collect', 'of', 'textil', 'sampl', 'lay', 'spread', 'out', 'on', 'the', 'tabl', 'sams

### Using `SnowballStemmer` <a name="stem-sno"></a>

In [ ]:
from nltk.stem import SnowballStemmer

sno = SnowballStemmer('english')

sno_stemms = [sno.stem(w) for w in re_tokens]
print(sno_stemms)

['one', 'morn', 'when', 'gregor', 'samsa', 'woke', 'from', 'troubl', 'dream', 'he', 'found', 'himself', 'transform', 'in', 'his', 'bed', 'into', 'a', 'horribl', 'vermin', 'he', 'lay', 'on', 'his', 'armour', 'like', 'back', 'and', 'if', 'he', 'lift', 'his', 'head', 'a', 'littl', 'he', 'could', 'see', 'his', 'brown', 'belli', 'slight', 'dome', 'and', 'divid', 'by', 'arch', 'into', 'stiff', 'section', 'the', 'bed', 'was', 'hard', 'abl', 'to', 'cover', 'it', 'and', 'seem', 'readi', 'to', 'slide', 'off', 'ani', 'moment', 'his', 'mani', 'leg', 'piti', 'thin', 'compar', 'with', 'the', 'size', 'of', 'the', 'rest', 'of', 'him', 'wave', 'about', 'helpless', 'as', 'he', 'look', 'what', 's', 'happen', 'to', 'me', 'he', 'thought', 'it', 'wasn', 't', 'a', 'dream', 'his', 'room', 'a', 'proper', 'human', 'room', 'although', 'a', 'littl', 'too', 'small', 'lay', 'peac', 'between', 'it', 'four', 'familiar', 'wall', 'a', 'collect', 'of', 'textil', 'sampl', 'lay', 'spread', 'out', 'on', 'the', 'tabl', 'sam

### Comparing Stemmers <a name="stem-compare"></a>

In [ ]:
sorted(list(set(ps_stemms) ^ set(sno_stemms)))

['A',
 'He',
 'It',
 'hard',
 'hardli',
 'helpless',
 'helplessli',
 'hi',
 'his',
 'slight',
 'slightli',
 'wa',
 'was']

`PorterStemmer` and `SnowballStemmer` handle adverbs differently